<h2><center><b><i>Cluster bomb</b></i>: Uncovering Patterns in Terrorist Group Beliefs and Attacks</center></h2>

#### **COM-480: Data Visualization**

**Team**: Alexander Sternfeld, Silvia Romanato & Antoine Bonnet

**Dataset**: [Global Terrorism Database (GTD)](https://www.start.umd.edu/gtd/) 

**Additional dataset**: [Profiles of Perpetrators of Terrorism in the United States (PPTUS)](https://dataverse.harvard.edu/dataset.xhtml?persistentId=hdl%3A1902.1/17702)

## **Terrorist ideologies**

In [9]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from load_data import *

pd.set_option('display.max_columns', None)

GTD = load_GTD()
PPTUS_data, PPTUS_sources = load_PPTUS()

GTD pickle file found, loading...
PPTUS pickle files found, loading...


### **Data preparation for stacked area charts**

We would like to create the following plot using D3.js: 

<p align="center">
  <img src="../plots/sketches/SketchArea.jpeg" 
  style="width: 500px; "/>
</p>

To do so, we prepare the data from both PPT-US and GTD datasets. We will create one data file with the annual number of attacks per main ideology. We will also create one data file per main ideology, with the annual number of attacks per sub-ideology. The user will scroll between each plot seamlessly. 

**Ideology labels**: Here are the categorical labels for each ideology and sub-ideology in the PPT-US, taken from the official Codebook: 

In [37]:
# Labels for dominant ideologies
DOM_labels = {
    'I_ETHNO': 'Ethno-nationalist',
    'I_REL': 'Religious',
    'I_RACE': 'Racist', 
    'I_LEFT': 'Extreme left',
    'I_RIGHT': 'Extreme right',
    'I_SI': 'Single issue',
}

# Labels for sub-ideologies
REL_labels = {'I_REL_1': 'Buddhist', 'I_REL_2': 'Christian/Catholic', 'I_REL_3': 'Cult', 'I_REL_4': 'Hindu', 'I_REL_5': 'Islamic (Sunni)', 'I_REL_6': 'Islamic (Shia)', 'I_REL_7': 'Jewish & Jewish Orthodox', 'I_REL_8': 'Sikh', 'I_REL_9': 'Pagan/Polytheist', 'I_REL_10': 'Occult/Satanist'}
RACE_labels = {'I_RACE_1': 'Anti-Semitic', 'I_RACE_2': 'Xenophobic/Nativist/Anti-Immigrant', 'I_RACE_3': 'Racial Supremacist/Racial Separatist'}
LEFT_labels = {'I_LEFT_1': 'Communist/Socialist', 'I_LEFT_2': 'Anarchists', 'I_LEFT_3': 'Marxist/Leninist', 'I_LEFT_4': 'Maoist', 'I_LEFT_5': 'Black Nationalist (US, Europe)', 'I_LEFT_6': 'New Left'}
RIGHT_labels = {'I_RIGHT_1': 'Anti-Semitic', 'I_RIGHT_2': 'Fascist/Neo-Fascist', 'I_RIGHT_3': 'Gun Rights', 'I_RIGHT_4': 'Tax Protest'}
SI_labels = {'I_SI_1': 'Anti-Abortion', 'I_SI_2': 'Anti-Catholic', 'I_SI_3': 'Anti-Communist', 'I_SI_4': 'Anti-Ecology/Pro-Development', 'I_SI_5': 'Anti-Homosexual/Anti-Sexual Deviance/Anti-Prostitution', 'I_SI_6': 'Anti-Nuclear', 'I_SI_7': 'Anti-Racist', 'I_SI_8': 'Anti-Religious', 'I_SI_9': 'Gun Rights', 'I_SI_10': 'Xenophobic/Nativist/Anti-Immigrant', 'I_SI_11': 'Tax Protest', 'I_SI_12': 'Anti-Castro', 'I_SI_13': 'Ecological/Anti-Technology/Primitivist/Animal Liberation', 'I_SI_14': '"Pacifist"/Anti-War'}

subIdeologies_pairs = {
    'Ethno-nationalist': {}, 
    'Religious': REL_labels,
    'Racist': RACE_labels, 
    'Extreme left': LEFT_labels,
    'Extreme right': RIGHT_labels,
    'Single issue': SI_labels
}

For all attacks perpetrated by organizations in the PPT-US, we merge terrorist attack data and group data. 

In [38]:
# Merge PPTUS and GTD 
print('Original number of attacks in GTD: ', GTD.shape[0])
df = PPTUS_data.merge(GTD, left_on='ORGNAME', right_on='gname', how= 'inner')
print('Number of attacks for which we know the organization ideology in GTD: ', df.shape[0])


Original number of attacks in GTD:  214666
Number of attacks for which we know the organization ideology in GTD:  7131


In [39]:
# Rename the dominant_ideology row from the number to the ideology name in the df dataframe
df['DOM_I'] = df['DOM_I'].replace(
    {1: 'Extreme right', 
     2: 'Extreme left', 
     3: 'Religious', 
     4: 'Ethno-nationalist', 
     5: 'Single issue', 
     -99: 'Uncertain'})

ethno_col = df.columns[df.columns.str.startswith('I_ETHNO') & ~df.columns.str.endswith('CI') & ~df.columns.str.endswith('SOURCE') & ~df.columns.str.endswith('I_ETHNO')]
rel_col = df.columns[df.columns.str.startswith('I_REL') & ~df.columns.str.endswith('CI') & ~df.columns.str.endswith('SOURCE') & ~df.columns.str.endswith('I_REL')]
race_col = df.columns[df.columns.str.startswith('I_RACE') & ~df.columns.str.endswith('CI') & ~df.columns.str.endswith('SOURCE') & ~df.columns.str.endswith('I_RACE')]
left_col = df.columns[df.columns.str.startswith('I_LEFT') & ~df.columns.str.endswith('CI') & ~df.columns.str.endswith('SOURCE') & ~df.columns.str.endswith('I_LEFT')]
right_col = df.columns[df.columns.str.startswith('I_RIGHT') & ~df.columns.str.endswith('CI') & ~df.columns.str.endswith('SOURCE') & ~df.columns.str.endswith('I_RIGHT')]
si_col = df.columns[df.columns.str.startswith('I_SI') & ~df.columns.str.endswith('CI') & ~df.columns.str.endswith('SOURCE') & ~df.columns.str.endswith('I_SI')]


In [41]:
# Create a column of Ideologies 
df['Ideologies'] = df.apply(lambda row: [DOM_labels[ideology_ID] for ideology_ID in DOM_labels.keys() if row[ideology_ID] == 1], axis=1)

# Create a column 'Sub-ideologies' that contains a list of sub-ideologies for which the corresponding I_TERMS is 1
df['Sub-ideologies'] = df.apply(lambda row: [REL_labels[sub_ideology] for sub_ideology in REL_labels.keys() if row[sub_ideology] == 1 and row['I_REL'] == 1] +
                                          [RACE_labels[sub_ideology] for sub_ideology in RACE_labels.keys() if row[sub_ideology] == 1 and row['I_RACE'] == 1] +
                                          [LEFT_labels[sub_ideology] for sub_ideology in LEFT_labels.keys() if row[sub_ideology] == 1 and row['I_LEFT'] == 1] +
                                          [RIGHT_labels[sub_ideology] for sub_ideology in RIGHT_labels.keys() if row[sub_ideology] == 1 and row['I_RIGHT'] == 1] +
                                          [SI_labels[sub_ideology] for sub_ideology in SI_labels.keys() if row[sub_ideology] == 1 and row['I_SI'] == 1],
                               axis=1)

In [74]:
df['Sub-ideologies']

0                                   [Communist/Socialist]
1                                       [Islamic (Sunni)]
2                                       [Islamic (Sunni)]
3                                       [Islamic (Sunni)]
4                                       [Islamic (Sunni)]
                              ...                        
7126    [Communist/Socialist, Marxist/Leninist, Maoist...
7127    [Communist/Socialist, Marxist/Leninist, Maoist...
7128                                        [Anti-Castro]
7129                                        [Anti-Castro]
7130                                                   []
Name: Sub-ideologies, Length: 7131, dtype: object

In [36]:
df['Sub-ideologies'] = df['Sub-ideologies'].apply(lambda x: x[0] if len(x) > 0 else None)

# Keep only columns ORGNAME, DOM_I 
#df = df[['iyear', 'ORGNAME', 'DOM_I', 'Ideologies', 'Sub-ideologies']].reset_index(drop=True)

# Format column names
df.rename(columns={'DOM_I': 'Dominant ideology', 'ORGNAME': 'Terrorist group', 'iyear': 'Year'}, inplace=True)

TypeError: object of type 'NoneType' has no len()

In [15]:
# Create the dominant ideologies datafile. Create a new dataframe with 1 row per year, and one column per dominant ideology. Values are the number of attacks of given dominant ideology per year
dom_table = df.groupby(['Year', 'Dominant ideology']).size().unstack(fill_value=0)

# Remove uncertain column
dom_table = dom_table.drop('Uncertain', axis=1)
dom_table.to_csv('../data/attacks_per_ideology/dominant_ideologies.csv')

sub_rel_table = df[df['Dominant ideology'] == 'Religious'][['Year', 'Sub-ideologies']].groupby(['Year', 'Sub-ideologies']).size().unstack(fill_value=0)
sub_rel_table = sub_rel_table.reindex(range(1970, 2018), fill_value=0)
sub_rel_table.to_csv('../data/attacks_per_ideology/sub_religious_ideologies.csv')

sub_ethno_table = df[df['Dominant ideology'] == 'Ethno-nationalist'][['Year', 'Sub-ideologies']].groupby(['Year', 'Sub-ideologies']).size().unstack(fill_value=0)
sub_ethno_table = sub_ethno_table.reindex(range(1970, 2018), fill_value=0)
sub_ethno_table.to_csv('../data/attacks_per_ideology/sub_ethnonat_ideologies.csv')

sub_right_table = df[df['Dominant ideology'] == 'Extreme right'][['Year', 'Sub-ideologies']].groupby(['Year', 'Sub-ideologies']).size().unstack(fill_value=0)
sub_right_table = sub_right_table.reindex(range(1970, 2018), fill_value=0)
sub_right_table.to_csv('../data/attacks_per_ideology/sub_right_ideologies.csv')

sub_left_table = df[df['Dominant ideology'] == 'Extreme left'][['Year', 'Sub-ideologies']].groupby(['Year', 'Sub-ideologies']).size().unstack(fill_value=0)
sub_left_table = sub_left_table.reindex(range(1970, 2018), fill_value=0)
sub_left_table.to_csv('../data/attacks_per_ideology/sub_left_ideologies.csv')

sub_SI_table = df[df['Dominant ideology'] == 'Single issue'][['Year', 'Sub-ideologies']].groupby(['Year', 'Sub-ideologies']).size().unstack(fill_value=0)
sub_SI_table = sub_SI_table.reindex(range(1970, 2018), fill_value=0)
sub_SI_table.to_csv('../data/attacks_per_ideology/sub_single_ideologies.csv')


In [16]:
# List of dominant ideologies
dominant_ideologies = df['Dominant ideology'].unique()

# Iterate over each dominant ideology
for ideology in dominant_ideologies:
    # Filter the data for the specific dominant ideology
    ideology_df = df[df['Dominant ideology'] == ideology]

    # Get all sub-ideologies for the dominant ideology
    sub_ideologies = subIdeologies_pairs[ideology]

    # Create a dataframe with one column per sub-ideology

KeyError: 'Uncertain'

In [4]:
# List of dominant ideologies
dominant_ideologies = df['Dominant ideology'].unique()

# Create a dictionary to store dataframes for each dominant ideology
dominant_ideology_dataframes = {}

for ideology in DOM_labels.values():
    # Filter the data for the specific dominant ideology
    ideology_df = df[df['Dominant ideology'] == ideology]

    # Create an empty dataframe to store the counts of sub-ideologies
    sub_ideology_counts = pd.DataFrame(columns=['Year'] + list(subIdeologies_pairs[ideology].values()))

    # Iterate over each year (convert to int to avoid float)
    start_year = int(ideology_df['Year'].min())
    end_year = int(ideology_df['Year'].max())
    for year in range(start_year, end_year + 1):
        # Filter the data for the specific year
        year_df = ideology_df[ideology_df['Year'] == year]

        # Create a dictionary to store the counts of sub-ideologies for the given year
        sub_ideology_count = {}

        # Iterate over each sub-ideology
        for sub_ideology_key, sub_ideology_label in subIdeologies_pairs[ideology].items():
            # Count the occurrences of the sub-ideology for the given year
            count = year_df['Sub-ideologies'].apply(lambda x: sub_ideology_label in x).sum()

            # Add the count to the dictionary
            sub_ideology_count[sub_ideology_label] = count

        # Add the counts to the sub_ideology_counts dataframe for the given year
        sub_ideology_counts = sub_ideology_counts.append({'Year': year, **sub_ideology_count}, ignore_index=True)

    # Set the Year column as the index
    sub_ideology_counts.set_index('Year', inplace=True)

    # Store the dataframe in the dictionary
    dominant_ideology_dataframes[ideology_label] = sub_ideology_counts

NameError: name 'df' is not defined

### **Data preparation for sunburst**


We would like to create the following plot using D3.js: 

<p align="center">
  <img src="../plots/sketches/SketchSunburst.jpeg" 
  style="width: 500px; "/>
</p>

We create a `.json` tree with three levels: Dominant ideology, attack type and target type. 

In [26]:
df.attacktype1_txt

0       Facility/Infrastructure Attack
1                    Bombing/Explosion
2                    Bombing/Explosion
3                    Bombing/Explosion
4                    Bombing/Explosion
                     ...              
7126                 Bombing/Explosion
7127    Facility/Infrastructure Attack
7128                 Bombing/Explosion
7129                 Bombing/Explosion
7130                 Bombing/Explosion
Name: attacktype1_txt, Length: 7131, dtype: object

In [17]:
df.rename(columns={'DOM_I': 'dominant_ideology'}, inplace=True)
# Keep only columns eventid, gname, attacktype1_txt, targtype1_txt and dominant_ideology
df = df[['eventid', 'gname', 'attacktype1_txt', 'targtype1_txt', 'dominant_ideology']]

# Remove all rows with uncertain dominant ideology
df = df[df['dominant_ideology'] != 'uncertain']

# Create a tree of dictionaries with keys {'name', 'children', 'value'} with layers ideology, attacktype1_txt, targtype1_txt from the df dataframe. Each layer has name, value = number of attacks and children = the next layer
tree = {'name': 'flare', 'children': []}

for ideology in df['dominant_ideology'].unique():
    df_ideology = df[df['dominant_ideology'] == ideology]
    layer1 = {'name': ideology}
    tree['children'].append(layer1)

    for attacktype in df_ideology['attacktype1_txt'].unique():
        df_attacktype = df_ideology[df_ideology['attacktype1_txt'] == attacktype]
        layer2 = {'name': attacktype}
        if 'children' in layer1.keys():
            layer1['children'].append(layer2)
        else: 
            layer1['children'] = [layer2]

# Save ideology_dict as json
with open('../maps/sunburst_tree.json', 'w') as fp:
    json.dump(tree, fp, indent=4)


KeyError: "None of [Index(['eventid', 'gname', 'attacktype1_txt', 'targtype1_txt',\n       'dominant_ideology'],\n      dtype='object')] are in the [columns]"

# Analysis 2:

### Plot 1.

In [ ]:
# plot the number of attacks per year per each dominant_ideology with a lineplot using plotly and fill the area under the line and 

fig = px.line(df.groupby(['iyear', 'dominant_ideology']).size().reset_index(name='counts'), x='iyear', y='counts', color='dominant_ideology', title='Number of attacks per year per each dominant_ideology', log_y=True)
fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    margin=dict(
        l = 50,
        r = 50,
        b = 100,
        t = 100,
        pad = 4
    ),
    paper_bgcolor="LightSteelBlue",
)
# fill the area under the line
#fig.update_traces(fill='tozeroy')
fig.show()


In [ ]:
# remove the -99 from the columns of df
df = df.replace(-99, 0)
df['I_ETHNO'].groupby([df['iyear']]).sum().reset_index().sort_values(by=['iyear'], ascending=True).rename(columns = {'iyear': 'year', 'I_ETHNO': 'ethno-nationalists'}).to_csv('../data/ETHNO.csv', index=False)
df[rel_col].groupby([df['iyear']]).sum().reset_index().sort_values(by=['iyear'], ascending=True).to_csv('../data/REL.csv', index=False)
df[race_col].groupby([df['iyear']]).sum().reset_index().sort_values(by=['iyear'], ascending=True).to_csv('../data/RACE.csv', index=False)
df[right_col].groupby([df['iyear']]).sum().reset_index().sort_values(by=['iyear'], ascending=True).to_csv('../data/RIGHT.csv', index=False)
df[left_col].groupby([df['iyear']]).sum().reset_index().sort_values(by=['iyear'], ascending=True).to_csv('../data/LEFT.csv', index=False)
df[si_col].groupby([df['iyear']]).sum().reset_index().sort_values(by=['iyear'], ascending=True).to_csv('../data/SI.csv', index=False)

In [ ]:
# print the df with rel_col and gname and iyear  and imonth
df[rel_col].groupby([df['iyear']]).sum().reset_index().sort_values(by=['iyear'], ascending=False)

# religious
fig = px.bar(df['I_ETHNO'].groupby([df['iyear']]).sum().reset_index().sort_values(by=['iyear'], ascending=False), x='iyear', y='I_ETHNO', title='Number of attacks per year per each ethno_nationalist/suprematist group', log_y=True)
# name the y axis as ethno_nationalist/suprematist
fig.update_yaxes(title_text='ethno_nationalist/suprematist')
fig.update_layout(autosize=False,width=1000,height=500,margin=dict(l = 50,r = 50,b = 100,t = 100,pad = 4),paper_bgcolor="LightSteelBlue")
fig.show()
# religious
fig = px.bar(df[rel_col].groupby([df['iyear']]).sum().reset_index().sort_values(by=['iyear'], ascending=False), x='iyear', y=rel_col, title='Number of attacks per year per each religious group', log_y=True)
fig.update_layout(autosize=False,width=1000,height=500,margin=dict(l = 50,r = 50,b = 100,t = 100,pad = 4),paper_bgcolor="LightSteelBlue")
sub_ideologies_religious = {'I_REL_1' : 'Buddhist', 'I_REL_2' : 'Catholic/Christian', 'I_REL_3' : 'Cult', 'I_REL_4' : 'Hindu', 'I_REL_5' : 'Islamic (Sunni)', 'I_REL_6' : 'Islamic (Shia)', 'I_REL_7' : 'Jewish/Jewish Orthodox', 'I_REL_8' : 'Sihk (Fundamentalists)', 'I_REL_9' : 'Pagan/Polytheists', 'I_REL_10' : 'Occult (including Satanist)'}
fig.for_each_trace(lambda t: t.update(name=sub_ideologies_religious[t.name]))
fig.show()
# racism
fig = px.bar(df[race_col].groupby([df['iyear']]).sum().reset_index().sort_values(by=['iyear'], ascending=False), x='iyear', y=race_col, title='Number of attacks per year per each racism group', log_y=True)
sub_ideologies_racism = {'I_RACE_1' : 'Anti-Semitic', 'I_RACE_2' : 'Xenophobic/Nativist/Anti-Immigrant', 'I_RACE_3' : 'Racial Supremacist/Racial Separatist'}
fig.update_layout(autosize=False,width=1000,height=500,margin=dict(l = 50,r = 50,b = 100,t = 100,pad = 4),paper_bgcolor="LightSteelBlue")
fig.for_each_trace(lambda t: t.update(name=sub_ideologies_racism[t.name]))
fig.show()
# right
fig = px.bar(df[right_col].groupby([df['iyear']]).sum().reset_index().sort_values(by=['iyear'], ascending=False), x='iyear', y=right_col, title='Number of attacks per year per each right group', log_y=True)
sub_ideologies_right= {'I_RIGHT_1': 'Anti-Semitic', 'I_RIGHT_2': 'Fascist/Neo-Fascist ', 'I_RIGHT_3': 'Gun Rights', 'I_RIGHT_4': 'Tax Protest'} 
fig.update_layout(autosize=False,width=1000,height=500,margin=dict(l = 50,r = 50,b = 100,t = 100,pad = 4),paper_bgcolor="LightSteelBlue")
fig.for_each_trace(lambda t: t.update(name=sub_ideologies_right[t.name]))
fig.show()
# left
fig = px.bar(df[left_col].groupby([df['iyear']]).sum().reset_index().sort_values(by=['iyear'], ascending=False), x='iyear', y=left_col, title='Number of attacks per year per each left group', log_y=True)
sub_ideologies_left= {'I_LEFT_1': 'Anti-Communist/Socialist', 'I_LEFT_2': 'Anarchists', 'I_LEFT_3': 'Marxist/Leninist', 'I_LEFT_4': 'Maoist', 'I_LEFT_5': 'Black Nationalist (US, Europe)', 'I_LEFT_6':'New Left'} 
fig.update_layout(autosize=False,width=1000,height=500,margin=dict(l = 50,r = 50,b = 100,t = 100,pad = 4),paper_bgcolor="LightSteelBlue")
fig.for_each_trace(lambda t: t.update(name=sub_ideologies_left[t.name]))
fig.show()
# single issue
fig = px.bar(df[si_col].groupby([df['iyear']]).sum().reset_index().sort_values(by=['iyear'], ascending=False), x='iyear', y=si_col, title='Number of attacks per year per each single issue group', log_y=True)
sub_ideologies_singleissue= {'I_SI_1': 'Anti-Abortion', 'I_SI_2': 'Anti-Catholic', 'I_SI_3': 'Anti-Communist', 'I_SI_4': 'Anti-Ecology/Pro-Development', 'I_SI_5': 'Anti-Homosexual/Anti-Sexual Deviance/Anti-Prostitution', 'I_SI_6': 'Anti-Nuclear', 'I_SI_7': 'Anti-Racist', 'I_SI_8': 'Anti-Religious', 'I_SI_9': 'Gun Rights', 'I_SI_10': 'Xenophobic/Nativist/Anti-Immigrant', 'I_SI_11': 'Tax Protest', 'I_SI_12': 'Anti-Castro', 'I_SI_13': 'Ecological/Anti-Technology/Primitivist/Animal Liberation', 'I_SI_14': '“Pacifist”/Anti-War' }
fig.update_layout(autosize=False,width=1000,height=500,margin=dict(l = 50,r = 50,b = 100,t = 100,pad = 4),paper_bgcolor="LightSteelBlue")
fig.for_each_trace(lambda t: t.update(name=sub_ideologies_singleissue[t.name]))
fig.show()


In [ ]:
# create an update menu to select the REL_COL OR ETHNO_COL OR RIGHT_COL OR LEFT_COL OR SI_COL
total_col = rel_col.append(si_col).append(right_col).append(left_col).append(race_col)
df_dict = df[total_col].groupby([df['iyear']]).sum().reset_index().sort_values(by=['iyear'], ascending=False)
fig = px.bar(df_dict, x='iyear', y=total_col, title='Number of attacks per year per each group', log_y=True)
fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    margin=dict(
        
        l = 50,
        r = 50,
        b = 100,
        t = 100,
        pad = 4
    ),
    paper_bgcolor="LightSteelBlue",
)
fig.update_layout(
    updatemenus=[
        dict(
            
            buttons=list([
                dict(
                    args= [{'y': [df_dict[col] for col in rel_col]}],
                    label="Religious",
                    method="update"
                ),
                dict(
                    args=[{'y': [df_dict[col] for col in race_col]}],
                    label="race",
                    method="update"
                ),
                dict(
                    args=[{'y': [df_dict[col] for col in right_col]}],
                    label="Right",
                    method="update"
                ),
                dict(
                    args=[{'y': [df_dict[col] for col in left_col]}],
                    label="Left",
                    method="update"
                ),
                dict(
                    args=[{'y': [df_dict[col] for col in si_col]}],
                    label="SI",
                    method="update"
                ),
                dict(
                    args=[{'y': [df_dict[col] for col in total_col]}],
                    label="All",
                    method="update"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)
fig.show()


does not work with plotly but since it's not necessary to implement it in plotly I will focus on how to do it in d3.

### Plot 2: check if the ideology is similar to the major goal

In [ ]:
ideologies = {1: "Extreme Right Wing", 2: "Extreme Left Wing",
              3: "Religious", 4: "Ethno-nationalist/separist", 5: "Single issue"}

ideology_counts = pd.DataFrame(columns=['ideology', 'count'])
for ideology in ideologies:
    ideology_counts = ideology_counts.append({'ideology': ideologies[ideology], 'count': len(
        df[df.dominant_ideology == ideology])}, ignore_index=True)

In [ ]:
ideologies = {1: "Extreme Right Wing", 2: "Extreme Left Wing",
              3: "Religious", 4: "Ethno-nationalist/separist", 5: "Single issue"}
ideology_structure = {}

for i in range(5):
    ideology = ideologies[i+1]
    subset = df[df.dominant_ideology == i+1]
    total_known_structure = len(subset[subset.G_POL_1 != -99])
    for j in range(6):
        pol_goal = j+1
        if j != 5:
            ideology_structure[(ideology, pol_goal)] = len(
                subset[subset['G_POL_'+str(pol_goal)] == 1])/total_known_structure
        if j ==5:
            ideology_structure[(ideology, pol_goal)] = len(
            subset[subset['G_POL_OTH'] == 1])/total_known_structure
ideology_structure_df = pd.DataFrame(columns=['ideology', 'pol_goal', 'percentage'])
for key in ideology_structure:
    ideology_structure_df = ideology_structure_df.append({'ideology': key[0], 'pol_goal': key[1], 'percentage': ideology_structure[key]}, ignore_index=True)

ideology_structure_df.set_index(['ideology', 'pol_goal'], inplace=True)

# Now make a grouped bar chart with the percentages. The chart should not be stacked, the bars should be next to each other
ideology_structure_df.unstack().plot(kind='bar', stacked=False, figsize=(10, 5))
# Change the legend. The legend should be outside the plot, for each color we should display the structure (1: Hierarchical, 2: Networked, 3: Cell-like, 4: Fragmented, 5: Other)
# Change the content of the legend
plt.legend(title="POLITICAL major goal", loc='center left', bbox_to_anchor=(1, 0.5), labels=['Protest government policies/laws', 'Protest government ruling party', 'Seek regime type', 'Territorial goals', 'Influence election', 'Other'])
plt.title("Group POLITICAL major goal per ideology")
plt.ylabel("Percentage")
plt.show()

In [ ]:
ideologies = {1: "Extreme Right Wing", 2: "Extreme Left Wing",
              3: "Religious", 4: "Ethno-nationalist/separist", 5: "Single issue"}
ideology_structure = {}

for i in range(5):
    ideology = ideologies[i+1]
    subset = df[df.dominant_ideology == i+1]
    total_known_structure = len(subset[subset.G_SOC_1 != -99])
    for j in range(3):
        soc_goal = j+1
        if j != 2:
            ideology_structure[(ideology, soc_goal)] = len(
                subset[subset['G_SOC_'+str(soc_goal)] == 1])/total_known_structure
        if j ==2:
            ideology_structure[(ideology, soc_goal)] = len(
            subset[subset['G_SOC_OTH'] == 1])/total_known_structure

ideology_structure_df = pd.DataFrame(columns=['ideology', 'soc_goal', 'percentage'])
for key in ideology_structure:
    ideology_structure_df = ideology_structure_df.append({'ideology': key[0], 'soc_goal': key[1], 'percentage': ideology_structure[key]}, ignore_index=True)

ideology_structure_df.set_index(['ideology', 'soc_goal'], inplace=True)

# Now make a grouped bar chart with the percentages. The chart should not be stacked, the bars should be next to each other
ideology_structure_df.unstack().plot(kind='bar', stacked=False, figsize=(10, 5))
# Change the legend. The legend should be outside the plot, for each color we should display the structure (1: Hierarchical, 2: Networked, 3: Cell-like, 4: Fragmented, 5: Other)
# Change the content of the legend
plt.legend(title="SOCIAL major goal", loc='center left', bbox_to_anchor=(1, 0.5), labels=['Protest social discrimination/social justice issues', 'Protest public & private institutions', 'Other'])
plt.title("Group SOCIAL major goal per ideology")
plt.ylabel("Percentage")
plt.show()

In [ ]:
ideologies = {1: "Extreme Right Wing", 2: "Extreme Left Wing",
              3: "Religious", 4: "Ethno-nationalist/separist", 5: "Single issue"}
ideology_structure = {}

for i in range(5):
    ideology = ideologies[i+1]
    subset = df[df.dominant_ideology == i+1]
    total_known_structure = len(subset[subset.G_ECO_1 != -99])
    for j in range(3):
        eco_goal = j+1
        if j != 2:
            ideology_structure[(ideology, eco_goal)] = len(
                subset[subset['G_ECO_'+str(eco_goal)] == 1])/total_known_structure
        if j ==2:
            ideology_structure[(ideology, eco_goal)] = len(
            subset[subset['G_ECO_OTH'] == 1])/total_known_structure

ideology_structure_df = pd.DataFrame(columns=['ideology', 'eco_goal', 'percentage'])
for key in ideology_structure:
    ideology_structure_df = ideology_structure_df.append({'ideology': key[0], 'eco_goal': key[1], 'percentage': ideology_structure[key]}, ignore_index=True)

ideology_structure_df.set_index(['ideology', 'eco_goal'], inplace=True)

# Now make a grouped bar chart with the percentages. The chart should not be stacked, the bars should be next to each other
ideology_structure_df.unstack().plot(kind='bar', stacked=False, figsize=(10, 5))
# Change the legend. The legend should be outside the plot, for each color we should display the structure (1: Hierarchical, 2: Networked, 3: Cell-like, 4: Fragmented, 5: Other)
# Change the content of the legend
plt.legend(title="ECONOMICAL major goal", loc='center left', bbox_to_anchor=(1, 0.5), labels=['Protest economic discrimination', 'Protest government/corporate economic policies', 'Other'])
plt.title("Group ECONOMICAL major goal per ideology")
plt.ylabel("Percentage")
plt.show()

In [ ]:
ideologies = {1: "Extreme Right Wing", 2: "Extreme Left Wing",
              3: "Religious", 4: "Ethno-nationalist/separist", 5: "Single issue"}
ideology_structure = {}

for i in range(5):
    ideology = ideologies[i+1]
    subset = df[df.dominant_ideology == i+1]
    total_known_structure = len(subset[subset.G_REL_1 != -99])
    for j in range(4):
        rel_goal = j+1
        if j != 3:
            ideology_structure[(ideology, rel_goal)] = len(
                subset[subset['G_REL_'+str(rel_goal)] == 1])/total_known_structure
        if j ==3:
            ideology_structure[(ideology, rel_goal)] = len(
            subset[subset['G_REL_OTH'] == 1])/total_known_structure

ideology_structure_df = pd.DataFrame(columns=['ideology', 'rel_goal', 'percentage'])
for key in ideology_structure:
    ideology_structure_df = ideology_structure_df.append({'ideology': key[0], 'rel_goal': key[1], 'percentage': ideology_structure[key]}, ignore_index=True)

ideology_structure_df.set_index(['ideology', 'rel_goal'], inplace=True)

# Now make a grouped bar chart with the percentages. The chart should not be stacked, the bars should be next to each other
ideology_structure_df.unstack().plot(kind='bar', stacked=False, figsize=(10, 5))
# Change the legend. The legend should be outside the plot, for each color we should display the structure (1: Hierarchical, 2: Networked, 3: Cell-like, 4: Fragmented, 5: Other)
# Change the content of the legend
plt.legend(title="RELIGIOUS major goal", loc='center left', bbox_to_anchor=(1, 0.5), labels=['Protest religious discrimination', 'Protest religious institutions', 'Killing infidels/Non-believers', 'Other'])
plt.title("Group RELIGIOUS major goal per ideology")
plt.ylabel("Percentage")
plt.show()

### Plot 3:

In [ ]:
MG_pol_col = df.columns[df.columns.str.startswith('G_POL') & ~df.columns.str.endswith('CI') & ~df.columns.str.endswith('SOURCE') & ~df.columns.str.endswith('TXT')]
MG_soc_col = df.columns[df.columns.str.startswith('G_SOC') & ~df.columns.str.endswith('CI') & ~df.columns.str.endswith('SOURCE')& ~df.columns.str.endswith('TXT')]
MG_eco_col = df.columns[df.columns.str.startswith('G_ECO') & ~df.columns.str.endswith('CI') & ~df.columns.str.endswith('SOURCE')& ~df.columns.str.endswith('TXT')]
MG_rel_col = df.columns[df.columns.str.startswith('G_REL') & ~df.columns.str.endswith('CI') & ~df.columns.str.endswith('SOURCE')& ~df.columns.str.endswith('TXT')] 

In [ ]:
total_col = MG_pol_col.append(MG_soc_col).append(MG_eco_col).append(MG_rel_col)
# count how many killed people happen in G_POL_1

# multiply the number of killed people with the number of attacks
df_dict = df[total_col].groupby([df['nkill']]).sum().reset_index().apply(lambda x: x['nkill']*x[total_col], axis=1).sum()

# plot a pie chart where the total is the sum of the number of killed people
fig = px.pie(df_dict, values=df_dict, names=total_col, title='Number of attacks per year per each group')
fig.show()


In [ ]:
# correlation between total_col
df[total_col].corr()
# plot the correlation
fig = px.imshow(df[total_col].corr())
fig.show()

In [ ]:
# plot the number of attacks per year per each group
total_col = MG_pol_col.append(MG_soc_col).append(MG_eco_col).append(MG_rel_col)

fig = px.bar(df[total_col].sum().reset_index(), x='index', y=0, title='Number of attacks per each major goal', log_y=False)
fig.update_layout(
    autosize=False,
    width=1000,
    height=500,
    margin=dict(
        
        l = 50,
        r = 50,
        b = 100,
        t = 100,
        pad = 4
    ),
    paper_bgcolor="LightSteelBlue",
)
fig.show()
        


# Analysis 1

THE DOMINANT IDEOLOGY CATEGORIES: <br>
-99= Uncertainty/conflicting information exists in available data<br> 
1= Extreme Right Wing (including all racist ideologies)<br>
2= Extreme Left Wing<br>
3= Religious<br>
4= Ethno-nationalist/Separatist<br>
5= Single Issue<br>

In [ ]:
# create five new dataframes, one for each ideology
df_ethno_nationalist = df[df['dominant_ideology'] == 4]
df_religious = df[df['dominant_ideology'] == 3]
df_extreme_right = df[df['dominant_ideology'] == 1]
df_extreme_left = df[df['dominant_ideology'] == 2]
df_single_issue = df[df['dominant_ideology'] == 5]
df_uncertain = df[df['dominant_ideology'] == -99]

print('The sum of the number of attacks of each ideology is equal to the total number of attacks: ', df_ethno_nationalist.shape[0] + df_religious.shape[0] + df_extreme_right.shape[0] + df_extreme_left.shape[0] + df_single_issue.shape[0] + df_uncertain.shape[0] == df.shape[0])

In [ ]:
# rename the dominant_ideology row from the number to the ideology name in the df dataframe
df['dominant_ideology'] = df['dominant_ideology'].replace({1: 'extreme_right', 2: 'extreme_left', 3: 'religious', 4: 'ethno_nationalist', 5: 'single_issue', -99: 'uncertain'})

## Analysis of the ethno_nationalists
1. geographic analysis
2. time series of attacks over time
3. number of groups and the most important ones
4. 

In [ ]:
fig1 = px.scatter_geo(df, lat = 'latitude', lon = 'longitude',animation_frame = 'dominant_ideology' , projection="natural earth")
fig1.update_layout(title = 'attacks')
fig1.show()

In [ ]:
df_ethno_nationalist.columns

In [ ]:
# do a time series graphh of the number of attacks per year for each ideology
df_ethno_nationalist['iyear'] = df_ethno_nationalist['iyear'].astype(int)

fig2 = px.line(df_ethno_nationalist.groupby('iyear').count().reset_index(), x = 'iyear', y = 'eventid')
fig2.update_layout(title = 'ethno_nationalist attacks')

df_religious['iyear'] = df_religious['iyear'].astype(int)

fig3 = px.line(df_religious.groupby('iyear').count().reset_index(), x = 'iyear', y = 'eventid')
fig3.update_layout(title = 'religious attacks')

df_extreme_left['iyear'] = df_extreme_left['iyear'].astype(int)

fig3 = px.line(df_extreme_left.groupby('iyear').count().reset_index(), x = 'iyear', y = 'eventid')
fig3.update_layout(title = 'etreme left attacks')

df_extreme_right['iyear'] = df_extreme_right['iyear'].astype(int)

fig3 = px.line(df_extreme_right.groupby('iyear').count().reset_index(), x = 'iyear', y = 'eventid')
fig3.update_layout(title = 'extreme right attacks')

df_single_issue['iyear'] = df_single_issue['iyear'].astype(int)

fig3 = px.line(df_single_issue.groupby('iyear').count().reset_index(), x = 'iyear', y = 'eventid')
fig3.update_layout(title = 'single issue attacks')

df_uncertain['iyear'] = df_uncertain['iyear'].astype(int)

fig3 = px.line(df_uncertain.groupby('iyear').count().reset_index(), x = 'iyear', y = 'eventid')
fig3.update_layout(title = 'uncertain attacks')

fig4 = make_subplots(rows=2, cols=3, subplot_titles=("ethno_nationalist attacks", "religious attacks", "extreme left attacks", "extreme right attacks", "single issue attacks", "uncertain attacks"))

fig4.add_trace(
    go.Scatter(x=df_ethno_nationalist.groupby('iyear').count().reset_index()['iyear'], y=df_ethno_nationalist.groupby('iyear').count().reset_index()['eventid'], name = 'ethno_nationalist'),
    row=1, col=1
)

fig4.add_trace(
    go.Scatter(x=df_religious.groupby('iyear').count().reset_index()['iyear'], y=df_religious.groupby('iyear').count().reset_index()['eventid'], name = 'religious'),
    row=1, col=2
)

fig4.add_trace(
    go.Scatter(x=df_extreme_left.groupby('iyear').count().reset_index()['iyear'], y=df_extreme_left.groupby('iyear').count().reset_index()['eventid'], name = 'extreme left'),
    row=2, col=1
)

fig4.add_trace(
    go.Scatter(x=df_extreme_right.groupby('iyear').count().reset_index()['iyear'], y=df_extreme_right.groupby('iyear').count().reset_index()['eventid'], name = 'extreme right'),
    row=2, col=2
)

fig4.add_trace(
    go.Scatter(x=df_single_issue.groupby('iyear').count().reset_index()['iyear'], y=df_single_issue.groupby('iyear').count().reset_index()['eventid'], name = 'single issue'),
    row=1, col=3
)

fig4.add_trace(
    go.Scatter(x=df_uncertain.groupby('iyear').count().reset_index()['iyear'], y=df_uncertain.groupby('iyear').count().reset_index()['eventid'], name = 'uncertain'),
    row=2, col=3
)

fig4.update_layout(height=600, width=1000, title_text="Subplots")
fig4.show()

In [ ]:
# get the top 3 weapons used by each ideology
print(df_ethno_nationalist['weaptype1_txt'].value_counts().head(3))
print(df_religious['weaptype1_txt'].value_counts().head(3))
print(df_extreme_left['weaptype1_txt'].value_counts().head(3))
print(df_extreme_right['weaptype1_txt'].value_counts().head(3))
print(df_single_issue['weaptype1_txt'].value_counts().head(3))
print(df_uncertain['weaptype1_txt'].value_counts().head(3))

In [ ]:
# diplay the number of killed and wounded people by each ideology per attack
print(df_ethno_nationalist[['nkill', 'nwound']].sum() / df_ethno_nationalist.shape[0])
print(df_religious[['nkill', 'nwound']].sum() / df_religious.shape[0])
print(df_extreme_left[['nkill', 'nwound']].sum() / df_extreme_left.shape[0])
print(df_extreme_right[['nkill', 'nwound']].sum() / df_extreme_right.shape[0])
print(df_single_issue[['nkill', 'nwound']].sum() / df_single_issue.shape[0])
print(df_uncertain[['nkill', 'nwound']].sum() / df_uncertain.shape[0])

In [ ]:
# plot a time series of the nuumber of killed people annd wounded people by each ideology per year on the same graph with different lines per ideology

fig5 = make_subplots(rows=1, cols=1, subplot_titles=("ethno_nationalist attacks"))

fig5.add_trace(
    go.Scatter(x=df_ethno_nationalist.groupby('iyear').sum().reset_index()['iyear'], y=df_ethno_nationalist.groupby('iyear').sum().reset_index()['nkill'], name = 'ethno_nationalist'),
    row = 1, col = 1
)

fig5.add_trace(
    go.Scatter(x=df_religious.groupby('iyear').sum().reset_index()['iyear'], y=df_religious.groupby('iyear').sum().reset_index()['nkill'], name = 'religious'),
    row = 1, col = 1
)

fig5.add_trace(
    go.Scatter(x=df_extreme_left.groupby('iyear').sum().reset_index()['iyear'], y=df_extreme_left.groupby('iyear').sum().reset_index()['nkill'], name = 'extreme left'),
    row = 1, col = 1
)

fig5.add_trace(
    go.Scatter(x=df_extreme_right.groupby('iyear').sum().reset_index()['iyear'], y=df_extreme_right.groupby('iyear').sum().reset_index()['nkill'], name = 'extreme right'),
    row = 1, col = 1
)

fig5.add_trace(
    go.Scatter(x=df_single_issue.groupby('iyear').sum().reset_index()['iyear'], y=df_single_issue.groupby('iyear').sum().reset_index()['nkill'], name = 'single issue'),
    row = 1, col = 1
)

fig5.show()


In [ ]:
to_save = df_ethno_nationalist[['ORGNAME', 'iyear']].groupby('iyear').count().reset_index().rename(columns = {'ORGNAME': 'ethno_nationalist'})
to_save['religious'] = df_religious[['ORGNAME', 'iyear']].groupby('iyear').count().reset_index()['ORGNAME']
to_save['extreme_left'] = df_extreme_left[['ORGNAME', 'iyear']].groupby('iyear').count().reset_index()['ORGNAME']
to_save['extreme_right'] = df_extreme_right[['ORGNAME', 'iyear']].groupby('iyear').count().reset_index()['ORGNAME']
to_save['single_issue'] = df_single_issue[['ORGNAME', 'iyear']].groupby('iyear').count().reset_index()['ORGNAME']
to_save['uncertain'] = df_uncertain[['ORGNAME', 'iyear']].groupby('iyear').count().reset_index()['ORGNAME']
to_save.head()


In [ ]:
to_save.to_csv('ethno_nationalist.csv', index = False)